In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import re # for regex pattern matching
import ase64 # for imaging

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})
                               
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
                               
#FIX ME Also remember to include a unique identifier such as your name or date
                               
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
               height = 250, width = 251))], href = 'https://www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('WBlackwell\' SNHU CS-340 Dashboard'))),
    html.Hr(),
    
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
        ],
        value='All'

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=True,
        row_selectable="single", #allows a row to be selected
        selected_rows=[],
        filter_action="native", #allows a filter
        sort_action="native", #allows sorting
        page_action="native", #enables pagination
        page_current=0, #set start page
        page_size=10, #set how many rows per page
    ),
    html.Br(),
    html.Hr(),
    
    
    
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content': 'center'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
   
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
                               
                               
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    
    if filter_type == 'All':
        df = pd.DataFrame.from_records(shelter.getRecordCriteria({})) 
                               
    elif filter_type == 'Water':
        #data that isn't clean: use regex for matching pattern
        #build/use the regex patterns for filters
        labRegex = re.compile(".*lab.*", re.IGNORECASE)
        chesaRegex = re.compile(".*chesa.*", re.IGNORECASE)
        newRegex = re.compile(".*newf.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.getRecordCriteria({
            #Regex not useable in an $in use $or
            '$or':[ 
                #pass regex to filter
                {"breed": {'$regex': newRegex}}, 
                {"breed": {'$regex': chesaRegex}},
                {"breed": {'$regex': labRegex}},
            ],
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0} #criteria
        }))
                               
    elif filter_type == 'Mountain':
        
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        alaskanRegex = re.compile(".*mala.*", re.IGNORECASE)
        oldRegex = re.compile(".*old engilish.*", re.IGNORECASE)
        huskyRegex = re.compile(".*husk.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.getRecordCriteria({
            '$or':[
                {"breed": {'$regex': rottRegex}},
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': alaskanRegex}},
                {"breed": {'$regex': huskyRegex}},
                {"breed": {'$regex': oldRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
                               
    elif filter_type == 'Disaster':
                               
        doberRegex = re.compile(".*dober.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        goldenRegex = re.compile(".*golden.*", re.IGNORECASE)
        bloodRegex = re.compile(".*blood.*", re.IGNORECASE)
        
        
        df = pd.DataFrame.from_records(shelter.getRecordCriteria({
            '$or':[
                {"breed": {'$regex': bloodRegex}},
                {"breed": {'$regex': doberRegex}},
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': goldenRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))
                               
    else: # return an exception to user
        raise Exception("Unknown filter")
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
                               
                               
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
                               
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
                               
                               
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    
    dffPie = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed',)
        )    
    ]                           

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    # geolocation: Hobart, Indiana = 41.529140 lat, -87.250328 long
    dff = pd.DataFrame.from_dict(viewData)
    return
        [
            dl.Map(style=('width': '1000px', 'height': '500px'), center = [41.529140, -87.250328], zoom=10, children=[
                dl.TileLayer(id="base.layer-id"),
                        dl. Marker(position=[41.53, -87.25], children=[
                            dl.Tooltip(dff.iloc[0,4]),
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(dff.iloc[1,9])
                            ])
                    ])
                ])
        ]


app  